## Introduction to Linear and Integer Programming

- **Definition:** Linear programming (LP) is a method to achieve the best outcome in a mathematical model whose requirements are represented by linear relationships. Integer programming (IP) is a specific case of LP where the solution space is restricted to integers.

- **Key Difference:** LP allows continuous decision variables, while IP restricts variables to discrete values (integers).

- **Applications:** These techniques are widely used in business for resource allocation, logistics, scheduling, and maximizing efficiency or profit.

### Components of a Linear Programming Model

- **Decision Variables:** These are the variables whose values are to be determined in order to optimize the objective function.

- **Objective Function:** A linear function to be maximized or minimized (e.g., maximize profit, minimize cost).

- **Constraints:** Linear inequalities or equalities representing restrictions or limitations on the decision variables (e.g., resource limits, capacity constraints).

Formulating a Linear Programming Problem

- **Identifying Decision Variables:** Understanding what quantities or factors need optimization.

- **Setting up the Objective Function:** Determining what is to be optimized (cost, profit, time, etc.).

- **Defining Constraints** Identifying and mathematically formulating limitations or requirements.

### Introduction to Integer Programming

- **Difference from LP:** In IP, decision variables are integers, which is crucial for problems where solutions are discrete (e.g., number of items, people).

- **Applications:** Used in situations where fractional answers are not practical (e.g., scheduling, manufacturing with batch restrictions).

### Problems

[1. Intro to Linear / Integer Programming](#1-intro-to-linear--integer-programming)\
[2. Resource Scheduling (Using `puLP`)](#2-resource-scheduling-using-pulp)\
[3. Getting started with `LpProblem()`](#3-getting-started-with-lpproblem)\
[4. Simple resource scheduling exercise](#4-simple-resource-scheduling-exercise)\
[5. Logistics planning problem](#5-logistics-planning-problem)\
[6. Example Scheduling Problem](#6-example-scheduling-problem)

In [1]:
import pandas as pd
import numpy as np
import warnings

from scipy.optimize import minimize
from pulp import *

warnings.filterwarnings("ignore")

### 1. Intro to Linear / Integer Programming

Linear programming involves:
1) Decision variables
2) Objective function
3) Constraints

Ex:

| | Pushup | Running |
|:-:|:-:| :-:|
|Minutes|0.2 per pushup|10 per mile|
|Calories|3 per pushup|130 per mile|

Constraint - only 10 minutes to exercise

1) Decision Variables:
- \# Pushups & \# Miles Ran

2) Obj. Function:
- $\text{maximize } 3P + 130M$
- where $P$ is the number of pushups and $M$ is the number miles to run.

3) Constraints:
- $0.2P + 10M \le 10$
- $P \ge 0$
- $M \ge 0$

In [2]:
# Define objective function
def obj_func(x):
    return (3 * x[0] + 130 * x[1])

# Construct bounds and constraints
bounds = [(0, None), (0, None)]
constraints = {"type": "ineq", "fun": lambda x: 10 - (.2 * x[0] + 10 * x[1])}

# Initial guess
i = [0, 0]

# Construct the optimization problem
opt = minimize(lambda x: -obj_func(x), i, bounds=bounds, constraints=constraints)
result = opt.x

print("do", round(result[0]), "pushups and run", round(result[1]), "miles")

do 50 pushups and run 0 miles


### 2. Resource Scheduling (Using `puLP`)

1. Sells $2$ types of cakes
2. $30$-day month
3. Available items:
    - $1$ oven
    - $2$ bakers
    - $1$ packaging packer - works only $22$ days.

Resource needs:

|     | Cake A | Cake B |
| :-: | :----: | :----: |
 Oven | 0.5 days | 1 day |
| Bakers | 1 day | 2.5 days |
| Packers | 1 day | 2 days |

<br>

|     | Cake A | Cake B |
| :-: | :----: | :----: |
|Profit|$20.00|$40.00|

**Objective Function**:

- $\text{maximize } 20A + 40B$

**Constraints**:

- $0.5A + B \le 30$
- $A + 2.5B \le 30 \times 2$
- $A + 2B \le 22$
- $A \ge 0$
- $B \ge 0$

In [3]:
def obj_func_2(x, pA, pB):
    return pA * x[0] + pB * x[1]

pA = 20
pB = 40
n_ovens = 1
n_bakers = 2
n_days_mo = 30
n_days_pkg = 22
n_packers = 1

i = [0, 0]
bounds = [(0, None) for _ in range(len(i))]
constraints = [
    # 30 * 1 - (0.5A + B) >= 0
    {"type": "ineq", "fun": lambda x: n_days_mo * n_ovens - (0.5 * x[0] + x[1])},
    # 30 * 2 - (A + 2.5B) >= 0
    {"type": "ineq", "fun": lambda x: n_days_mo * n_bakers - (x[0] + 2.5 * x[1])},
    # 22 * 1 - (A + 2B) >= 0
    {"type": "ineq", "fun": lambda x: n_days_pkg * n_packers - (x[0] + 2 * x[1])}
]

opt = minimize(
    lambda n_cakes: -obj_func_2(n_cakes, pA=20, pB=40), 
    i, 
    bounds=bounds, 
    constraints=constraints
)

opt.x

array([4.4, 8.8])

In [4]:
m = LpProblem("Maximize Bakery Profits", LpMaximize)
A = LpVariable("A", lowBound=0, cat="Integer")
B = LpVariable("B", lowBound=0, cat="Integer")
m += pA * A + pB * B
m += 0.5 * A + 1 * B <= n_days_mo
m += 1 * A + 2.5 * B <= n_days_mo * n_bakers
m += 1 * A + 2 * B <= n_days_pkg * n_packers

m.solve()
print(f"Produce {A.varValue:,.0f} of Cake A, and {B.varValue:,.0f} of Cake B.")

Produce 0 of Cake A, and 11 of Cake B.


### 3. Getting started with LpProblem()

You have been given the role of scheduler for a job shop. A job shop is a type of manufacturing process in which small batches of a variety of custom products are made. A poorly scheduled job shop will increase working capital and reduce the total number of jobs completed. These situations negatively affect a company's overall operations. You are looking to optimize the schedule.

How could you model this in PuLP?

1. Initialize `LpProblem()` with `LpMinimize`, and define the objective function as the number of jobs completed.

2. Initialize `LpProblem()` with `LpMaximize`, and define the objective function as the number of jobs not completed.

3. <u>***Initialize `LpProblem()` with `LpMinimize`, and define the objective function as the number of jobs not completed.***</u>

4. Initialize `LpProblem()` with `LpMaximize`, and define the objective function as the amount of working capital.

### 4. Simple resource scheduling exercise

<i>You are planning the production at a glass manufacturer. This manufacturer only produces wine and beer glasses:

- there is a maximum production capacity of 60 hours

- each batch of wine and beer glasses takes 6 and 5 hours respectively

- the warehouse has a maximum capacity of 150 rack spaces

- each batch of the wine and beer glasses takes 10 and 20 spaces respectively

- the production equipment can only make full batches, no partial batches

Also, we only have orders for 6 batches of wine glasses. Therefore, we do not want to produce more than this. Each batch of the wine glasses earns a profit of $5 and the beer $4.5.

The objective is to maximize the profit for the manufacturer.</i>

**Solution**:

Since we are only able to make *full* batches, this is an *integer programming* problem. Our problem is defined as:

\begin{align*}
\text{maximize} \quad & 5W + 4.5B\\
\text{subject to} \quad & 6W + 5B \le 60 \\
                        & 10W + 20B \le 150 \\
                        & 0 \le W \le 6 \\
                        & B \ge 0
\end{align*}


In [5]:
m = LpProblem("wine_beer_manufacturing", LpMaximize)

W = LpVariable("wine_batches", lowBound=0, upBound=6, cat="Integer")
B = LpVariable("beer_batchces", lowBound=0, cat="Integer")

m += 5 * W + 4.5 * B
m += 6 * W + 5 * B <= 60
m += 10 * W + 20 * B <= 150

m.solve()

print(f"Produce {W.varValue:,.0f} batches of wine, and {B.varValue:,.0f} batches of beer.")

Produce 6 batches of wine, and 4 batches of beer.


### 5. Logistics planning problem
<i>You are consulting for kitchen oven manufacturer helping to plan their logistics for next month. There are two warehouse locations (New York, and Atlanta), and four regional customer locations (East, South, Midwest, West). The expected demand next month for East it is 1,800, for South it is 1,200, for the Midwest it is 1,100, and for West it is 1000. The cost for shipping each of the warehouse locations to the regional customer's is listed in the table below. Your goal is to fulfill the regional demand at the lowest price.</i>

|Customer|	New York|	Atlanta|
|:-:|:-:|:-:|
|East|	$211|	$232|
|South|	$232|	$212
|Midwest|	$240|	$230|
|West|	$300|	$280|

**Solution**:

\begin{align*}
& N \rightarrow \text{Shipments from the New York warehouse} \\
& A \rightarrow \text{Shipments from the Atlanta warehouse} \\
\\
\text{minimize}   \quad & C = 211N_E + 232A_E + 232N_S + 212A_S + 240N_M + 230A_M + 300N_W + 280A_W \\
\text{subject to} \quad &N_E + A_E = 1800 \\
                  \quad & N_S + A_S = 1200 \\
                  \quad & N_M + A_M = 1100 \\
                  \quad & N_W + A_W = 1000 \\
                  \quad & N_E,N_S,N_M,N_W,A_E,A_S,A_M,A_W \geq 0

\end{align*}

In [6]:
# Define variables
warehouses = ["New York", "Atlanta"]
regions = ["East", "South", "Midwest", "West"]
demand = [1800, 1200, 1100, 1000]
costs_list = [211, 232, 240, 300, 232, 212, 230, 280]
regional_demand = dict(zip(regions, demand))
warehouse_regions = [(w, r) for w in warehouses for r in regions]
costs = {wr: c for wr, c in zip(warehouse_regions, costs_list)}

var_dict = {
    k: LpVariable(f"{k[0][0]}{k[1][0]}", 0, None, "Integer") for k in costs.keys()
}

# Initialize model
m = LpProblem("minimize_costs", LpMinimize)

# Define objective function
m += lpSum([costs[wr] * var_dict[wr] for wr in warehouse_regions])

for r in regions:
    m += lpSum([var_dict[(w, r)] for w in warehouses]) == regional_demand[r]

m.solve()

if m.status == 1:
    shipments_ = {wr: v.varValue for wr, v in var_dict.items()}
    shipments = pd.DataFrame()
    for (w, r), v in shipments_.items():
        shipments.loc[w, r] = v
    print(shipments.T.applymap(lambda x: f"{x:,.0f}"), "\n")
    print(f"Total Cost: ${m.objective.value():,.2f}")
else:
    print("Optimization failed.")

        New York Atlanta
East       1,800       0
South          0   1,200
Midwest        0   1,100
West           0   1,000 

Total Cost: $1,167,200.00


### 6. Example Scheduling Problem

|Day of Week|Index|Drivers Needed|
|:-:|:-:|:-:|
|Monday|0|11
|Tuesday|1|14
Wednesday|2|23
Thursday|3|21
Friday|4|20
Saturday|5|15
Sunday|6|8

How many drivers, in total, do we need to hire?

Constraint:
Each driver for 5 consecutive days, followed by 2 days off, repeated weekly.

<b><i>Proposed</i> Solution</b>:

\begin{align*}
\text{minimize}     \quad & T = \sum^{6}_{i=0}X_i \\
\text{subject to}   \quad & X_0 + X_1 + X_2 + X_3 + X_4 \ge 20 \\
                    \quad & X_1 + X_2 + X_3 + X_4 + X_5 \ge 15 \\
                    \quad & X_2 + X_3 + X_4 + X_5 + X_6 \ge 8  \\
                    \quad & X_3 + X_4 + X_5 + X_6 + X_0 \ge 11 \\
                    \quad & X_4 + X_5 + X_6 + X_0 + X_1 \ge 14 \\
                    \quad & X_5 + X_6 + X_0 + X_1 + X_2 \ge 23 \\
                    \quad & X_6 + X_0 + X_1 + X_2 + X_3 \ge 21 \\
\end{align*}

In [7]:
days = list(range(7))
drivers_needed = [11, 14, 23, 21, 20, 15, 8]

m = LpProblem("driver_optimization", LpMinimize)

x = LpVariable.dicts("drivers", days, lowBound=0, cat="Integer")

m += lpSum([x[i] for i in days])

m += x[0] + x[1] + x[2] + x[3] + x[4] >= 20
m += x[1] + x[2] + x[3] + x[4] + x[5] >= 15
m += x[2] + x[3] + x[4] + x[5] + x[6] >= 8
m += x[3] + x[4] + x[5] + x[6] + x[0] >= 11
m += x[4] + x[5] + x[6] + x[0] + x[1] >= 14
m += x[5] + x[6] + x[0] + x[1] + x[2] >= 23
m += x[6] + x[0] + x[1] + x[2] + x[3] >= 21

m.solve()

if m.status == 1:
    drivers = pd.DataFrame(
        {day: [drivers.varValue] for day, drivers in x.items()}
    ).T.reset_index()
    
    drivers.columns = ["Day", "Drivers Needed"]
    drivers = drivers.set_index("Day")
    print(pd.DataFrame(drivers).T.applymap(lambda x: f"{x:,.0f}").T)

    Drivers Needed
Day               
0                8
1                8
2                4
3                1
4                0
5                3
6                0


**DOES NOT MATCH REQUIREMENTS!**